# Model Serving 

In notebook '04-04-accident-recognition' we were able to use the retrained model to predict a 'severe' or 'moderate' car accident within an image.  

Now we will determine if we can query the model directly from the model server we have created.  This will be done through an API call.

**Important**: First, enter the inference endpoint URL you got after deploying the model server.

In [ ]:
# Normally, this should be
# RestURL = 'http://modelmesh-serving.userX:8008' , with userX being replaced by the user you have been assigned.
# CHANGE the value below, or nothing will work!
RestURL = 'https://famodel-sandbox.apps.rhoai.sandbox2386.opentlc.com'

In [ ]:
# If you did not use the Workbench image designed for this Lab, you can uncomment and run the following line to install the required packages.
#!pip install --upgrade pip
!pip install --no-cache-dir --no-dependencies -r requirements.txt
!pip install --no-cache-dir --no-dependencies opencv-python

In [ ]:
import cv2
from matplotlib import pyplot as plt
from PIL import Image
## note that to keep the notebook readable, a lot of code was moved into the file remote_infer.py.
## if you are curious, open it to see the various functions required around this prediction
from remote_infer import process_image

## We will define the inference URL, the model name, the YAML file with your classes

In [ ]:
infer_url = f'{RestURL}/v2/models/famodel/versions/1/infer'

## Now we set the parameters for the inference

In [ ]:
# 1. The image you want to analyze
image_path = '/opt/app-root/src/data/16.png'  # You can replace this with an image you upload

## Launch the inference and show the result

In [ ]:
import requests
import numpy as np
from PIL import Image
import json
from transformers import ViTImageProcessor
import warnings

# Suppress the InsecureRequestWarning for cleaner output
warnings.filterwarnings("ignore", message="Unverified HTTPS request is being made.*")

# --- 1. SET UP YOUR VARIABLES ---

# The base URL for your deployed model server
RestURL = 'https://famodel-sandbox.apps.rhoai.sandbox2386.opentlc.com'

# The full URL for your specific model and version
INFERENCE_URL = f'{RestURL}/v2/models/famodel/versions/1/infer'

# This dictionary maps the model's output index to the age group label.
id2label = {0: '01', 1: '02', 2: '03', 3: '04', 4: '05', 5: '06-07', 6: '08-09', 7: '10-12', 8: '13-15', 9: '16-20', 10: '21-25', 11: '26-30', 12: '31-35', 13: '36-40', 14: '41-45', 15: '46-50', 16: '51-55', 17: '56-60', 18: '61-65', 19: '66-70', 20: '71-80', 21: '81-90', 22: '90+'}

# --- THIS IS THE CORRECTED LINE ---
# Load the original processor config from the Hugging Face Hub
path_to_processor = "dima806/facial_age_image_detection"

# The path to an image you want to test
image_path = '/opt/app-root/src/data/16.png' 

# --- 2. LOAD AND PREPROCESS YOUR IMAGE ---

print(f"Loading and preparing image: {image_path}")
try:
    # Load the processor to prepare the image exactly as the model expects
    processor = ViTImageProcessor.from_pretrained(path_to_processor)
    
    # Load the test image
    image = Image.open(image_path).convert("RGB")
    display(image)

    # The processor handles resizing, normalization, etc.
    inputs = processor(images=image, return_tensors="np")
    image_data = inputs['pixel_values'].astype(np.float32)

    # --- 3. BUILD THE CORRECT PAYLOAD ---
    
    payload = {
        "inputs": [
            {
                "name": "pixel_values",
                "shape": image_data.shape,
                "datatype": "FP32",
                "data": image_data.tolist()
            }
        ]
    }

    # --- 4. SEND THE REQUEST AND GET THE RESULT ---

    print(f"Sending request to the model server...")
    response = requests.post(INFERENCE_URL, json=payload, verify=False)
    response.raise_for_status() 
    
    result = response.json()
    
    output_data = np.array(result['outputs'][0]['data'])
    predicted_class_index = np.argmax(output_data)
    predicted_age_group = id2label[predicted_class_index]
    
    print("\n✅ Success!")
    print(f"Predicted Age Group: {predicted_age_group}")

except FileNotFoundError:
    print(f"❌ ERROR: The image file was not found at '{image_path}'. Please check the path and try again.")
except Exception as e:
    print(f"\n❌ An error occurred during inference: {e}")
    if 'response' in locals() and hasattr(response, 'text'):
        print("--- Server Response ---")
        print(response.text)